## PAIR PROGRAMMING LIMPIEZA IV

### Valores Nulos
---

En este caso trabajaremos con el dataframe que limpiamos en el ejercicio de pair de Limpieza anterior, el de Limpieza III, donde limpiabamos las columnas de species, age, etc. y eliminabamos los outliers.
​

Antes de seguir, recordamos las preguntas que nos planteamos al principio del pair programming de EDA para dirigir nuestro análisis.

- ¿Es Australia es el sitio más peligroso y letal para estar relajada en la playa?
- ¿Cuál es el rango de edad que sufre la mayoría de los ataques?
- Independientemente de la edad, sufren los hombres más ataques que las mujeres?
- ¿En qué mes ocurren más ataques?
- ¿Cuál es la relación entre la especie y el tipo de ataque (si es fatal o no)?
- ¿Cómo han evolucionado los ataques a lo largo del tiempo?

Actualizamos esta tabla ya que en el ejercicio de pair de Limpieza I y II ya dejamos algunas columnas limpitas:


- age ✔️ la limpiamos en el pair de Limpieza II
- species ✔️ la limpiamos en el pair de Limpieza II
- country ✔️ la limpiamos en el pair de Limpieza II
- fatal ✔️ la limpiamos en el pair de Pandas V
- year ✔️ es una columna de tipo float deberíamos convertirla a integer
- sex ✔️ la limpiamos en el pair de Pandas V
- fecha ✔️ la limpiamos en el pair de Pandas V

Es el momento de ponernos a trabajar con los valores nulos 💪🏽.

A lo largo de este ejercicio de pair programming vamos a intentar eliminar los valores nulos de nuestras columnas. En la lección hemos aprendido varios métodos, nosotras os planteamos los ejercicios pero sentiros libres de usar el método que más se adapte a vuestras necesidades. Manos a la obra!

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sidetable

In [2]:
df = pd.read_pickle("../files/attacks10.pickle")
df.sample(5)

,case_number,year,type,country,activity,age,species_,date,mes_ataque,fatal,sexo,cat_species,edades,mes
867,2002.07.20,2002,Unprovoked,usa,Swimming,15,"Bull shark, 1.8 m to 2.1 m [6' to 7']",20-Jul-2002,Jul,N,F,Bull,15.0,Jul
310,2015.04.12,2015,Unprovoked,reunion,Surfing,13,Bull shark,12-Apr-2015,Apr,Y,M,Bull,13.0,Apr
1520,1959.10.04,1959,Unprovoked,usa,Diving for abalone,30,"White shark, 5 m [16.5'], identified by Dr. W....",04-Oct-1959,Oct,N,M,White,30.0,Oct
55,ND-0148,1997,Unprovoked,usa,Swimming,NaN,NaN,2009?,NaN,N,M,NaN,NaN,NaN
1555,1957.04.13,1957,Unprovoked,australia,Swimming between anchored pearling luggers,22,Shark seen feeding on turtle scraps thrown ove...,13-Apr-1957,Apr,N,M,Unspecified,22.0,Apr


In [39]:
#df['rango_edad']= pd.cut(df['edades'], 3 , labels = ['Jovenes', 'Adultos', 'Ancianos'])
#df.head()

**EJERCICIO 1**

Lo primero que tenemos que evaluar es en que columnas tenemos nulos y que cantidad tenemos en cada una. ¿Hay alguna columna con una gran cantidad de nulos? En caso de que sea así deberemos eliminarla.

In [40]:
# Comprobamos la cantidad total y porcentaje de valores nulos del DataFrame con el método stb.missing
df.stb.missing().round(3)

,missing,total,percent
mes,170,1672,10.167
edades,162,1672,9.689
age,154,1672,9.211
mes_ataque,141,1672,8.433
species_,126,1672,7.536
cat_species,126,1672,7.536
fatal,97,1672,5.801
activity,31,1672,1.854
sexo,14,1672,0.837
country,10,1672,0.598


    Vemos que las columnas que tienen nulos no superan el 10% del total de los datos,  por lo que consideramos mas conveniente reemplazar los valores nulos por una nueva categoria para seguir teniendo el registro de esos ataques en vez de eliminarlos.

In [ ]:
# Almacenamos en un diccionario los índices de las filas que tienen valores nulos iterando por cada columna
indices = [] 
for i in df.columns:
    ind = list(df[df[i].isnull()].index)
    indices.extend(ind)
indices

In [ ]:
# Convertimos el diccionario en set para eliminar los duplicados
indices = list(set(indices))
indices

In [43]:
# Creamos un DataFrame con las filas que contienen valores nulos
df_nulos = df.iloc[indices]
df_nulos

,case_number,year,type,country,activity,age,species_,date,mes_ataque,fatal,sexo,cat_species,edades,mes
0,1800.00.00,1997,Unprovoked,seychelles,a corsair's boat was overturned,NaN,NaN,1800,NaN,Y,F,NaN,NaN,NaN
1,1797.05.28.R,1997,Unprovoked,NaN,Dropped overboard,NaN,NaN,Reported May-28-1797,May,Y,NaN,NaN,NaN,May
2,1792.09.12,1997,Provoked,england,Fishing,NaN,NaN,12-Sep-1792,Sep,Y,M,NaN,NaN,Sep
3,1791.00.00,1997,Unprovoked,australia,NaN,NaN,NaN,1791,NaN,Y,F,NaN,NaN,NaN
4,1788.05.10,1997,Boat,australia,Fishing,NaN,NaN,10-May-1788,May,N,NaN,NaN,NaN,May
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,2011.01.20,2011,Invalid,australia,Swimming,7,Shark involvement not confirmed,20-Jan-2011,Jan,NaN,F,Unspecified,7.0,Jan
1521,1959.10.00,1959,Unprovoked,fiji,Spearfishing,36,"Tiger shark, 1.8 m [6']",Oct-1959,Oct,N,M,Tiger,36.0,NaN
1523,1959.08.20,1959,Invalid,philippines,The 240-ton motor vessel Pilar II with 100 peo...,10,Shark involvement prior to death was not confi...,20-Aug-1959,Aug,NaN,M,Unspecified,10.0,Aug
1020,1997.02.28.a,1997,Unprovoked,australia,Scuba diving,30,"Tiger shark, 2m to 3m",28-Feb-1997,Feb,N,M,Tiger,NaN,Feb


---

**EJERCICIO 2**

Es el momento de eliminar los nulos:

- Reemplazad los valores nulos de la columna age por la media de la edad, redondeada a dos decimales.


In [45]:
# Utilizamos el método .fillna
df['edades'].fillna(df['edades'].mean().round(2), inplace=True)

In [46]:
df['edades'].isnull().sum() # Comprobamos que no hay mas nulos en la columna 'edades'

0

- En relación a la columna de country al tratarse de una columna de tipo categórica, reemplazad los valores nulos por una nueva categória que se llame Unknown.


In [47]:
# Utilizamos el método. replace
df['country'].replace(np.nan, 'Unknow', inplace=True)

In [48]:
df['country'].isnull().sum() # Comprobamos que no hay mas nulos en la columna 'country'

0

- Reemplazad los valores nulos de la columna fatal por Unknown.

In [50]:
# Utilizamos el método .replace
df['fatal'].replace(np.nan, 'Unknow', inplace=True)

In [51]:
df['fatal'].isnull().sum() # Comprobamos que no hay mas nulos en la columna 'fatal'

0

- Reemplazad los valores nulos de la columna type por el valor más frecuente (la moda).


In [53]:
# Utilizamos el método .replace
df['type'].replace(np.nan, df.type.mode()[0], inplace=True)

In [54]:
df['type'].isnull().sum() # Comprobamos que no hay mas nulos en la columna 'type'

0

- Reemplazad los valores nulos de la columna fecha por Unknown.


In [55]:
# Utilizamos el método .replace
df['mes_ataque'].replace(np.nan, 'Unknow', inplace= True)

In [56]:
df['mes_ataque'].isnull().sum() # Comprobamos que no hay mas nulos en la columna 'mes_ataque'

0

In [59]:
df.stb.missing() # Comprobamos con el método stb.missing que no hay valores nulos en las cinco columnas modificadas en el ejercicio anterior

,missing,total,percent
mes,170,1672,10.167464
age,154,1672,9.210526
species_,126,1672,7.535885
cat_species,126,1672,7.535885
activity,31,1672,1.854067
sexo,14,1672,0.837321
case_number,0,1672,0.000000
year,0,1672,0.000000
type,0,1672,0.000000
country,0,1672,0.000000


---

**EJERCICIO 3**

Guardad el csv para seguir trabajando con el en los siguientes ejercicios de pair.


In [57]:
df.to_pickle('../files/attacks11.pickle')